In [7]:
import sqlite3
import os
import re

# Open folder and directory
data = 'C:\\Users\\Wren\\Documents\\THEtool_logfiles'
filenames = []
dir_list = os.listdir(data)
for file in dir_list:
    if re.match(r"\d+-\w+-\S+", file):
        filenames.append(file)

# Empty a table in the log
for file in filenames:
    conn = sqlite3.connect(os.path.join(data, file))
    c = conn.cursor()
    c.execute("DELETE FROM props;")
    conn.commit()
    
# Clear tabs 
for file in filenames:
    conn = sqlite3.connect(os.path.join(data, file))
    c.execute("UPDATE act SET json='{\"is_focused\":false}' where k='focus' AND json LIKE '%is_focused\":false%'")
    conn.commit()

# Find the lowest value of t for each file
lowest_t = {}
for file in filenames:
    conn = sqlite3.connect(os.path.join(data, file))
    c = conn.cursor()
    c.execute("SELECT MIN(t) FROM (SELECT MIN(t) AS t FROM act UNION SELECT MIN(t) AS t FROM key UNION SELECT MIN(t) AS t FROM eye);")
    row = c.fetchone()
    if row is not None:
        lowest_t[file] = row[0]
    conn.close()

# Subtract the lowest t from all t values in each file
for file, lowt in lowest_t.items():
    conn = sqlite3.connect(os.path.join(data, file))
    c = conn.cursor()
    c.execute("UPDATE act SET t = t - ? WHERE t >= ?;", (lowt, lowt))
    c.execute("UPDATE key SET t = t - ? WHERE t >= ?;", (lowt, lowt))
    c.execute("UPDATE eye SET t = t - ? WHERE t >= ?;", (lowt, lowt))
    conn.commit()

# Change the name of the log
for file in filenames:   
    current_name = os.path.join(data, file)
    new_name = re.sub(r"(\d+-)\w+(-\S+)", r"\1anon\2", file)
    try:
        os.rename(current_name, os.path.join(data, new_name))
    except Exception as e:
        print(f"Skipping {current_name} - unable to rename file ({e})")
        continue

# Commit the changes and close the connection
conn.commit()
conn.close()

Skipping C:\Users\Wren\Documents\THEtool_logfiles\7000-iytxeq7u-xMB1FzNEjz - unable to rename file ([WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Wren\\Documents\\THEtool_logfiles\\7000-iytxeq7u-xMB1FzNEjz' -> 'C:\\Users\\Wren\\Documents\\THEtool_logfiles\\7000-anon-xMB1FzNEjz')


In [8]:
import sqlite3
import os
import re

data = 'C:\\Users\\Wren\\Documents\\THEtool_logfiles'
filenames_anon = []
dir_list = os.listdir(data)
for file in dir_list:
    if re.match(r"\d+-anon-\S+", file):
        filenames_anon.append(file)
        
for file in filenames_anon:
    conn = sqlite3.connect(os.path.join(data, file))
    c = conn.cursor()
    c.execute("UPDATE act SET json='{\"is_focused\":false}' where k='focus' AND json LIKE '%is_focused\":false%'")
    conn.commit()